In [ ]:
import re
import glob
import pandas as pd
import nibabel as nb
import numpy as np
from nistats.second_level_model import SecondLevelModel
from nistats.thresholding import map_threshold
from nilearn import plotting


# importing rmaps

In [ ]:
network_interest = "DMN"
connectivity_maps = glob.glob("rest/rmap_seeds/*sub*_"+network_interest+".nii.gz")
len(connectivity_maps)

# import csv file 

In [ ]:
cimaq_diagnostic = 'cimaq_diagnostic_Nov_2019.csv'
cimaq_diagnostic = pd.read_csv(cimaq_diagnostic)

print(set(cimaq_diagnostic['V01 36448_diagnostic_clinique,36448_diagnostic_clinique'].values))
cimaq_diagnostic = cimaq_diagnostic[['V01 demographics,CandID','V01 36448_diagnostic_clinique,36448_diagnostic_clinique']]
cimaq_diagnostic.rename(columns = {"V01 demographics,CandID":"subID","V01 36448_diagnostic_clinique,36448_diagnostic_clinique":"diagnostic" },inplace=True)
cimaq_diagnostic.shape

# create design matrix

In [ ]:

cimaq_diagnostic["MCI"] = 0
cimaq_diagnostic["SCD"] = 0
cimaq_diagnostic["CN"] = 0

subjectID_set = set([sub.split("sub")[1].split("_")[0] for sub in connectivity_maps])
drop = []
for i,r in cimaq_diagnostic.iterrows():
    
    #
    if(r["diagnostic"] == "troubles_subjectifs_de_cognition" ):
        cimaq_diagnostic.loc[i,"SCD"] = 1

    if(r["diagnostic"] == "cognitivement_sain_(cs)"):
        cimaq_diagnostic.loc[i,"CN"] = 1          

    if("trouble_cognitif_léger" in r["diagnostic"] ):
        cimaq_diagnostic.loc[i,"MCI"] = 1


    cimaq_diagnostic.loc[i,"subID"] = str(r["subID"])
    

cimaq_diagnostic = cimaq_diagnostic.loc[cimaq_diagnostic["diagnostic"] != "démence_de_type_alzheimer-légère" ]
cimaq_diagnostic = cimaq_diagnostic.loc[cimaq_diagnostic["subID"].isin(subjectID_set)]
cimaq_diagnostic.head()

In [ ]:
design_matrix = cimaq_diagnostic[["MCI","SCD","CN"]]
design_matrix.shape

# perform  t-test

In [ ]:
second_level_input = []
for i in connectivity_maps:
    sub = re.search("\d\d\d\d\d\d",i)
    subid = sub.group(0)
    if(subid in set(cimaq_diagnostic["subID"].values)):        
        second_level_input.append(i)


In [ ]:
second_level_model = SecondLevelModel().fit(second_level_input, design_matrix=design_matrix)
p_map = second_level_model.compute_contrast(second_level_contrast = [0,-1,1])

In [ ]:
plotting.view_img(p_map)

In [ ]:
thresholded_map, threshold = map_threshold(p_map, alpha=0.5, height_control='fdr')
plotting.view_img(thresholded_map)

# F-test

In [ ]:
second_level_model = SecondLevelModel().fit(second_level_input, design_matrix=design_matrix)
p_map = second_level_model.compute_contrast(second_level_contrast = [1,1,1], second_level_stat_type = "F")

In [ ]:
plotting.view_img(p_map)